In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,38018
2,Huelva,Confirmados PDIA 14 días,2757
3,Huelva,Tasa PDIA 14 días,"537,2488648985716"
4,Huelva,Confirmados PDIA 7 días,1334
5,Huelva,Tasa PDIA 7 dias,"259,9528421380829"
6,Huelva,Total Confirmados,38225
7,Huelva,Curados,33439
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  38018.0


/tmp/ipykernel_93475/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  10625.0


/tmp/ipykernel_93475/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_93475/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_93475/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_93475/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 364 personas en los últimos 7 días 

Un positivo PCR cada 165 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,38018.0,1334.0,2757.0,513170.0,259.952842,537.248865,283.0
Huelva-Costa,22339.0,831.0,1782.0,289548.0,286.999047,615.441999,167.0
Huelva (capital),10625.0,395.0,874.0,143837.0,274.616406,607.632250,78.0
Condado-Campiña,11926.0,328.0,631.0,156231.0,209.945529,403.889113,62.0
Sierra de Huelva-Andévalo Central,3379.0,174.0,342.0,67391.0,258.194714,507.486163,53.0
Isla Cristina,2590.0,81.0,177.0,21393.0,378.628523,827.373440,22.0
Punta Umbría,1033.0,73.0,133.0,15355.0,475.415174,866.167372,20.0
Lepe,2482.0,72.0,168.0,27880.0,258.249641,602.582496,16.0
Trigueros,394.0,28.0,42.0,7862.0,356.143475,534.215212,15.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,150.0,24.0,32.0,1310.0,1832.061069,2442.748092,2.0
Corteconcepción,32.0,8.0,8.0,536.0,1492.537313,1492.537313,6.0
Santa Ana la Real,18.0,2.0,7.0,475.0,421.052632,1473.684211,2.0
Beas,291.0,26.0,49.0,4341.0,598.940336,1128.772172,5.0
Cumbres Mayores,74.0,15.0,19.0,1749.0,857.632933,1086.335049,0.0
Punta Umbría,1033.0,73.0,133.0,15355.0,475.415174,866.167372,20.0
Isla Cristina,2590.0,81.0,177.0,21393.0,378.628523,827.373440,22.0
Palos de la Frontera,869.0,39.0,95.0,11742.0,332.141032,809.061489,3.0
Nerva,228.0,15.0,41.0,5169.0,290.191526,793.190172,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Bartolomé de la Torre,321.0,1.0,6.0,3761.0,26.588673,159.532039,1.0,0.166667
Alosno,212.0,1.0,4.0,3933.0,25.425884,101.703534,1.0,0.250000
Aroche,101.0,2.0,8.0,3054.0,65.487885,261.951539,0.0,0.250000
Bonares,354.0,7.0,26.0,6060.0,115.511551,429.042904,0.0,0.269231
Niebla,316.0,3.0,11.0,4116.0,72.886297,267.249757,0.0,0.272727
Campillo (El),82.0,2.0,7.0,2024.0,98.814229,345.849802,0.0,0.285714
Zarza-Perrunal (La),41.0,2.0,7.0,1253.0,159.616919,558.659218,1.0,0.285714
Santa Ana la Real,18.0,2.0,7.0,475.0,421.052632,1473.684211,2.0,0.285714
Lucena del Puerto,299.0,5.0,17.0,3261.0,153.327200,521.312481,2.0,0.294118
